* Python code replication of: " https://www.kaggle.com/janniskueck/pm1-notebook1-prediction-newdata "
* Created by: Alexander Quispe

This notebook contains an example for teaching.

# Introduction

In labor economics an important question is what determines the wage of workers. This is a causal question, but we could begin to investigate from a predictive perspective.

In the following wage example,  Y  is the hourly wage of a worker and  X  is a vector of worker's characteristics, e.g., education, experience, gender. Two main questions here are:

* How to use job-relevant characteristics, such as education and experience, to best predict wages?

* What is the difference in predicted wages between men and women with the same job-relevant characteristics?

In this lab, we focus on the prediction question first.

# Data

The data set we consider is from the March Supplement of the U.S. Current Population Survey, year 2015. We select white non-hispanic individuals, aged 25 to 64 years, and working more than 35 hours per week during at least 50 weeks of the year. We exclude self-employed workers; individuals living in group quarters; individuals in the military, agricultural or private household sectors; individuals with inconsistent reports on earnings and employment status; individuals with allocated or missing information in any of the variables used in the analysis; and individuals with hourly wage below  3 .

The variable of interest  Y  is the hourly wage rate constructed as the ratio of the annual earnings to the total number of hours worked, which is constructed in turn as the product of number of weeks worked and the usual number of hours worked per week. In our analysis, we also focus on single (never married) workers. The final sample is of size  **n=5150** .

# Data analysis


We start by loading the data set.

In [2]:
# Import relevant packages
import pandas as pd
import numpy as np
#import pyreadr

In [3]:
#rdata_read = pyreadr.read_r("../data/wage2015_subsample_inference.Rdata")
#data = rdata_read[ 'data' ]

data  = pd.read_csv(r'../../../data/wage2015_subsample_inference.csv')

data['occ']=pd.Categorical(data.occ)
data['occ2']=pd.Categorical(data.occ2)
data['ind']=pd.Categorical(data.ind)
data['ind2']=pd.Categorical(data.ind2)


type(data)
data.shape

(5150, 21)

Let's have a look at the structure of the data.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5150 entries, 0 to 5149
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   rownames  5150 non-null   int64   
 1   wage      5150 non-null   float64 
 2   lwage     5150 non-null   float64 
 3   sex       5150 non-null   float64 
 4   shs       5150 non-null   float64 
 5   hsg       5150 non-null   float64 
 6   scl       5150 non-null   float64 
 7   clg       5150 non-null   float64 
 8   ad        5150 non-null   float64 
 9   mw        5150 non-null   float64 
 10  so        5150 non-null   float64 
 11  we        5150 non-null   float64 
 12  ne        5150 non-null   float64 
 13  exp1      5150 non-null   float64 
 14  exp2      5150 non-null   float64 
 15  exp3      5150 non-null   float64 
 16  exp4      5150 non-null   float64 
 17  occ       5150 non-null   category
 18  occ2      5150 non-null   category
 19  ind       5150 non-null   category
 20  ind2    

***Variable description***

- occ : occupational classification
- ind : industry classification
- lwage : log hourly wage
- sex : gender (1 female) (0 male)
- shs : some high school
- hsg : High school graduated
- scl : Some College
- clg: College Graduate
- ad: Advanced Degree
- ne: Northeast
- mw: Midwest
- so: South
- we: West
- exp1: experience

In [7]:
data

,rownames,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,...,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
0,10,9.615385,2.263364,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,7.0,0.49,0.343,0.2401,3600.0,11,8370.0,18
1,12,48.076923,3.872802,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,31.0,9.61,29.791,92.3521,3050.0,10,5070.0,9
2,15,11.057692,2.403126,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,18.0,3.24,5.832,10.4976,6260.0,19,770.0,4
3,18,13.942308,2.634928,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,25.0,6.25,15.625,39.0625,420.0,1,6990.0,12
4,19,28.846154,3.361977,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,22.0,4.84,10.648,23.4256,2015.0,6,9470.0,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5145,32620,14.769231,2.692546,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,9.0,0.81,0.729,0.6561,4700.0,16,4970.0,9
5146,32624,23.076923,3.138833,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,12.0,1.44,1.728,2.0736,4110.0,13,8680.0,20
5147,32626,38.461538,3.649659,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,11.0,1.21,1.331,1.4641,1550.0,4,3680.0,6
5148,32631,32.967033,3.495508,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,10.0,1.00,1.000,1.0000,2920.0,9,6570.0,11


In [4]:
data.describe()

,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4
count,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000,5150.000000
mean,23.410410,2.970787,0.444466,0.023301,0.243883,0.278058,0.317670,0.137087,0.259612,0.296505,0.216117,0.227767,13.760583,3.018925,8.235867,25.118038
std,21.003016,0.570385,0.496955,0.150872,0.429465,0.448086,0.465616,0.343973,0.438464,0.456761,0.411635,0.419432,10.609465,4.000904,14.488962,53.530225
min,3.021978,1.105912,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13.461538,2.599837,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.250000,0.125000,0.062500
50%,19.230769,2.956512,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,1.000000,1.000000,1.000000
75%,27.777778,3.324236,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,21.000000,4.410000,9.261000,19.448100
max,528.845673,6.270697,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,47.000000,22.090000,103.823000,487.968100


We are constructing the output variable  **Y**  and the matrix  **Z**  which includes the characteristics of workers that are given in the data.

In [13]:
Y = np.log2(data['wage']) 
n = len(Y)
z = data.loc[:, ~data.columns.isin(['wage', 'lwage','Unnamed: 0'])]
p = z.shape[1]

print("Number of observation:", n, '\n')
print( "Number of raw regressors:", p)

Number of observation: 5150 

Number of raw regressors: 19


For the outcome variable *wage* and a subset of the raw regressors, we calculate the empirical mean to get familiar with the data.

In [14]:
Z_subset = data.loc[:, data.columns.isin(["lwage","sex","shs","hsg","scl","clg","ad","mw","so","we","ne","exp1"])]
table = Z_subset.mean(axis=0)
table

lwage     2.970787
sex       0.444466
shs       0.023301
hsg       0.243883
scl       0.278058
clg       0.317670
ad        0.137087
mw        0.259612
so        0.296505
we        0.216117
ne        0.227767
exp1     13.760583
dtype: float64

In [16]:
table = pd.DataFrame(data=table, columns={"Sample mean":"0"} )
table.index
index1 = list(table.index)
index2 = ["Log Wage","Sex","Some High School","High School Graduate",\
          "Some College","College Graduate", "Advanced Degree","Midwest",\
          "South","West","Northeast","Experience"]

In [17]:
table

,Sample mean
lwage,2.970787
sex,0.444466
shs,0.023301
hsg,0.243883
scl,0.278058
clg,0.317670
ad,0.137087
mw,0.259612
so,0.296505
we,0.216117


In [18]:
table = table.rename(index=dict(zip(index1,index2)))
table

,Sample mean
Log Wage,2.970787
Sex,0.444466
Some High School,0.023301
High School Graduate,0.243883
Some College,0.278058
College Graduate,0.317670
Advanced Degree,0.137087
Midwest,0.259612
South,0.296505
West,0.216117


E.g., the share of female workers in our sample is ~44% ($sex=1$ if female).

Alternatively, we can also print the table as latex.

In [19]:
print(table.to_latex())

\begin{tabular}{lr}
\toprule
{} &  Sample mean \\
\midrule
Log Wage             &     2.970787 \\
Sex                  &     0.444466 \\
Some High School     &     0.023301 \\
High School Graduate &     0.243883 \\
Some College         &     0.278058 \\
College Graduate     &     0.317670 \\
Advanced Degree      &     0.137087 \\
Midwest              &     0.259612 \\
South                &     0.296505 \\
West                 &     0.216117 \\
Northeast            &     0.227767 \\
Experience           &    13.760583 \\
\bottomrule
\end{tabular}



## Prediction Question

Now, we will construct a prediction rule for hourly wage $Y$, which depends linearly on job-relevant characteristics $X$:

\begin{equation}\label{decompose}
Y = \beta'X+ \epsilon.
\end{equation}

Our goals are

* Predict wages  using various characteristics of workers.

* Assess the predictive performance using the (adjusted) sample MSE, the (adjusted) sample $R^2$ and the out-of-sample MSE and $R^2$.


We employ two different specifications for prediction:


1. Basic Model:   $X$ consists of a set of raw regressors (e.g. gender, experience, education indicators,  occupation and industry indicators, regional indicators).


2. Flexible Model:  $X$ consists of all raw regressors from the basic model plus occupation and industry indicators, transformations (e.g., ${exp}^2$ and ${exp}^3$) and additional two-way interactions of polynomial in experience with other regressors. An example of a regressor created through a two-way interaction is *experience* times the indicator of having a *college degree*.

Using the **Flexible Model**, enables us to approximate the real relationship by a
 more complex regression model and therefore to reduce the bias. The **Flexible Model** increases the range of potential shapes of the estimated regression function. In general, flexible models often deliver good prediction accuracy but give models which are harder to interpret.

Now, let us fit both models to our data by running ordinary least squares (ols):

In [20]:
# Import packages for OLS regression
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [21]:
# 1. basic model
basic = 'lwage ~ sex + exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2'
basic_results = smf.ols(basic , data=data).fit()
print(basic_results.summary()) # estimated coefficients
print( "Number of regressors in the basic model:",len(basic_results.params), '\n')  # number of regressors in the Basic Model

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.310
Model:                            OLS   Adj. R-squared:                  0.303
Method:                 Least Squares   F-statistic:                     45.83
Date:                Thu, 31 Mar 2022   Prob (F-statistic):               0.00
Time:                        23:38:25   Log-Likelihood:                -3459.9
No. Observations:                5150   AIC:                             7022.
Df Residuals:                    5099   BIC:                             7356.
Df Model:                          50                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.7222      0.080     46.330      0.0

##### Note that the basic model consists of $51$ regressors.

In [22]:
# 2. flexible model
flex = 'lwage ~ sex + shs+hsg+scl+clg+occ2+ind2+mw+so+we + (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+occ2+ind2+mw+so+we)'
flex_results_0 = smf.ols(flex , data=data)
flex_results = smf.ols(flex , data=data).fit()
print(flex_results.summary()) # estimated coefficients
print( "Number of regressors in the basic model:",len(flex_results.params), '\n') # number of regressors in the Flexible Model

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.351
Model:                            OLS   Adj. R-squared:                  0.319
Method:                 Least Squares   F-statistic:                     10.83
Date:                Thu, 31 Mar 2022   Prob (F-statistic):          2.69e-305
Time:                        23:38:49   Log-Likelihood:                -3301.9
No. Observations:                5150   AIC:                             7096.
Df Residuals:                    4904   BIC:                             8706.
Df Model:                         245                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           3.8603      0.429     

Note that the flexible model consists of $246$ regressors.

## Try Lasso next

In [23]:
# Import relevant packages for lasso 
#from sklearn.linear_model import LassoCV
#from sklearn import linear_model
#from sklearn.preprocessing import PolynomialFeatures
#from sklearn.metrics import mean_squared_error

In [24]:
# Get exogenous variables from flexible model
#X = flex_results_0.exog
#X.shape

In [32]:
# Set endogenous variable
lwage = data["lwage"]
lwage.shape

(5150,)

In [26]:
#alpha=0.1

In [27]:
# Set penalty value = 0.1
#reg = linear_model.Lasso(alpha=0.1/np.log(len(lwage)))
#reg = linear_model.Lasso(alpha = alpha)

# LASSO regression for flexible model
#reg.fit(X, lwage)
#lwage_lasso_fitted = reg.fit(X, lwage).predict( X )

# coefficients 
#reg.coef_
#print('Lasso Regression: R^2 score', reg.score(X, lwage))

In [28]:
# Check predicted values
#lwage_lasso_fitted

Now, we can evaluate the performance of both models based on the (adjusted) $R^2_{sample}$ and the (adjusted) $MSE_{sample}$:

In [29]:
# Basic Model
basic = 'lwage ~ sex + exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2'
basic_results = smf.ols(basic , data=data).fit()

# Flexible model 
flex = 'lwage ~ sex + shs+hsg+scl+clg+occ2+ind2+mw+so+we + (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+occ2+ind2+mw+so+we)'
flex_results = smf.ols(flex , data=data).fit()

In [30]:
# Assess the predictive performance
R2_1 = basic_results.rsquared
print("R-squared for the basic model: ", R2_1, "\n")
R2_adj1 = basic_results.rsquared_adj
print("adjusted R-squared for the basic model: ", R2_adj1, "\n")


R2_2 = flex_results.rsquared
print("R-squared for the flexible model: ", R2_2, "\n")
R2_adj2 = flex_results.rsquared_adj
print("adjusted R-squared for the flexible model: ", R2_adj2, "\n")

#R2_L = reg.score(flex_results_0.exog, lwage)
#print("R-squared for LASSO: ", R2_L, "\n")
#R2_adjL = 1 - (1-R2_L)*(len(lwage)-1)/(len(lwage)-X.shape[1]-1)
#print("adjusted R-squared for LASSO: ", R2_adjL, "\n")

R-squared for the basic model:  0.31004650692219493 

adjusted R-squared for the basic model:  0.3032809304064291 

R-squared for the flexible model:  0.3511098950617233 

adjusted R-squared for the flexible model:  0.31869185352218865 



In [33]:
# calculating the MSE
MSE1 =  np.mean(basic_results.resid**2)
print("MSE for the basic model: ", MSE1, "\n")
p1 = len(basic_results.params) # number of regressors
n = len(lwage)
MSE_adj1  = (n/(n-p1))*MSE1
print("adjusted MSE for the basic model: ", MSE_adj1, "\n")

MSE2 =  np.mean(flex_results.resid**2)
print("MSE for the flexible model: ", MSE2, "\n")
p2 = len(flex_results.params) # number of regressors
n = len(lwage)
MSE_adj2  = (n/(n-p2))*MSE2
print("adjusted MSE for the flexible model: ", MSE_adj2, "\n")


#MSEL = mean_squared_error(lwage, lwage_lasso_fitted)
#print("MSE for the LASSO model: ", MSEL, "\n")
#pL = reg.coef_.shape[0] # number of regressors
#n = len(lwage)
#MSE_adjL  = (n/(n-pL))*MSEL
#print("adjusted MSE for LASSO model: ", MSE_adjL, "\n")

MSE for the basic model:  0.22442505581164462 

adjusted MSE for the basic model:  0.22666974650519117 

MSE for the flexible model:  0.21106813644318204 

adjusted MSE for the flexible model:  0.22165597526149827 



In [36]:
# Package for latex table 
#import array_to_latex as a2l

#table = np.zeros((2, 5))
#table[0,0:5] = [p1, R2_1, MSE1, R2_adj1, MSE_adj1]
#table[1,0:5] = [p2, R2_2, MSE2, R2_adj2, MSE_adj2]
#table[2,0:5] = [pL, R2_L, MSEL, R2_adjL, MSE_adjL]
#table

In [38]:
#table = pd.DataFrame(table, columns = ["p","$R^2_{sample}$","$MSE_{sample}$","$R^2_{adjusted}$", "$MSE_{adjusted}$"], \
                      #index = ["basic reg","flexible reg"])
#table